In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [3]:
import glob
import tensorflow as tf
import cv2
import numpy as np
import skvideo.io
from tqdm.auto import tqdm
from skimage import transform
from ganime.visualization.videos import display_videos
from ganime.visualization.images import display_images
from joblib import Parallel, delayed

In [4]:
source_folder = "../../../data/kny/scenes/01/*.mp4"

In [5]:
video_paths = sorted(glob.glob(source_folder))

In [6]:
def resize_video(video, new_height=64, new_width=128):
    n_frames = video.shape[0]
    frames = np.zeros((n_frames, new_height, new_width, 3))
    for i in range(n_frames):
        frame = video[i]
        resized = cv2.resize(frame,(new_width, new_height),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        frames[i] = resized
    return frames

In [7]:
def frame_count(video_path, manual=False):
    def manual_count(handler):
        frames = 0
        while True:
            status, frame = handler.read()
            if not status:
                break
            frames += 1
        return frames 

    cap = cv2.VideoCapture(video_path)
    # Slow, inefficient but 100% accurate method 
    if manual:
        frames = manual_count(cap)
    # Fast, efficient but inaccurate method
    else:
        try:
            frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        except:
            frames = manual_count(cap)
    cap.release()
    return frames

In [8]:
all_frame_numbers = Parallel(n_jobs=-1)(delayed(frame_count)(path) for path in tqdm(video_paths))
all_frame_numbers = np.array(all_frame_numbers)

  0%|          | 0/381 [00:00<?, ?it/s]

In [9]:
def get_value_to_split(array):
    n_elements = array.shape[0]
    range_min = 15
    range_max = 25
    
    lowest_remainder = range_max
    lowest_id = range_max
    for i in range(range_min, range_max):
        remainder = n_elements % i
        if remainder <= lowest_remainder:
            lowest_id = i
            lowest_remainder = remainder
    return lowest_id

In [10]:
def cut_array(array, number):
    n_splits = (array.shape[0] // number) 
    if n_splits == 0:
        yield array
    else:
        for i in range(n_splits):
            start = i * number
            end = (i+1) * number
            if array.shape[0] - end < number:
                end = array.shape[0]
            yield array[start:end]

In [11]:
def load_video(path):
    videos = []
    video = skvideo.io.vread(path)
    video = resize_video(video)
    video = video.astype(np.uint8)
    for video_split in cut_array(video, get_value_to_split(video)):
        videos.append(video_split)
    #videos = np.array(videos, dtype=object)
    videos = tf.ragged.stack([tf.convert_to_tensor(video) for video in videos], axis=0)
    return videos
    

In [12]:
videos = Parallel(n_jobs=1)(delayed(load_video)(path) for path in tqdm(video_paths))
videos = tf.concat(videos, axis=0)

  0%|          | 0/381 [00:00<?, ?it/s]

2022-07-12 00:12:31.633489: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-12 00:12:31.995984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22297 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:a1:00.0, compute capability: 8.6


In [13]:
videos.shape

TensorShape([1506, None, None, 128, 3])